In [ ]:
import pandas as pd
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_files = r"/content/drive/MyDrive/Data_M/1M_Min_clusters/"

### **Récupérer les différents fichiers de clusters**

In [ ]:
# récupérer les fichiers de clusters
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_km= glob.glob(f"{path_files}*km*")
list_som= glob.glob(f"{path_files}*som*")

In [ ]:
list_km

['/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_50',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_100']

In [ ]:
list_som

['/content/drive/MyDrive/Data_M/1M_Min_clusters/som_225',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/som_100',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/som_400']

### **Récupérer et trier les fichiers de tweets bruts des fichiers de tweets nettoyés**

In [ ]:
# vérifier la liste
liste_clean_km=[]
liste_brut_km=[]
for cluster in list_km:
  liste_clean_km.append(glob.glob(cluster+'/*clean*'))
  liste_brut_km.append(glob.glob(cluster+'/*raw*'))

In [ ]:
liste_clean_km[0]

['/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_2_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_7_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_6_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_0_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_1_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_5_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_3_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_4_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_20_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_8_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_25/km_25_cluster_22_clean_msg.txt',
 '/content/drive/MyDrive/Data_M/1M_Min_clusters/km_2

On remarque que les différents clusters ne sont pas ordonnés

In [ ]:
""" fonction pour lire une liste de fichier et mettre leur contenu dans une liste
# argument : une liste de fichiers
# sortie: liste"""
def read_cluster(files):
  liste=[]
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      liste.append(file.readlines())
  return liste

In [ ]:
def read_cluster(files):
  liste=[]
  for i,cluster in enumerate(files):
      liste.append(pd.read_fwf(cluster, names= ['tweet']))
  return liste

In [ ]:
def matrice(liste):
  cluster_km=[]
  for file in liste:
    file.sort()
    cluster_km.append(read_cluster(file))
  return cluster_km

Comparaison entre deux façon de lecture des données dataframe et matrice

In [ ]:
%%time
clean_cluster_km= matrice(liste_clean_km)
brut_cluster_km= matrice(liste_brut_km)

CPU times: user 2.86 s, sys: 698 ms, total: 3.56 s
Wall time: 4.17 s


La lecture de données par readlines est beaucoup plus rapide qu'avec pandas

In [ ]:
# Mettre le contenu de chaque cluster dans un dataframe
df=[[],[],[]]
for i in range(len(brut_cluster_km)):
  for n in range(len(brut_cluster_km[i])):
      df[i].append(pd.DataFrame(list(zip(brut_cluster_km[i][n] ,clean_cluster_km[i][n])), columns = ['tweet', 'clean_twt']))

In [ ]:
df[2][99]

,tweet,clean_twt
0,#NowPlaying EDWIGE MARIE https://t.co/LBvnfxCJ...,#nowplaying edwige marie\n
1,charles?!.. charles &lt;3☆ charles?!☆♡ charles...,charles?! .. charles & lt;3 charles?! charles ...
2,avisa louis\n,avisa louis\n
3,Siemens #Class374 4030 working ES9008 0801 Lon...,siemens #class374 4030 working es9008 0801 lon...
4,Tom Holland para GQ Magazine 2021. #SpiderManN...,tom holland para gq magazine 2021. #spidermann...
...,...,...
1995,Paul Torres-Cohen $C1JS.F\n,paul torres-cohen c1js.f\n
1996,YVES NAAAAUUURRRRR 😭😭😭\n,yves naaaauuurrrrr\n
1997,2021 GQ Tom holland&gt;&gt;&gt;&gt;&gt;&gt; ht...,2021 gq tom holland&gt;&gt;&gt;&gt;&gt;&gt;\n
1998,Charles Pétillon https://t.co/SlkxA0KdWF\n,charles pétillon\n


In [ ]:
!pip install gensim==4.0
!pip install python-Levenshtein

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
# récupérer les embeddings entrainés sur nos données
w2v_min = KeyedVectors.load('/content/drive/MyDrive/Data_M/w2v/w2vec_model_d300_1M_Min')

In [ ]:
import numpy as np

In [ ]:
#encode un message
def encode(msg):
    return np.mean([w2v_min[word] for word in msg if word in w2v_min] or [np.zeros(300)], axis = 0)

In [ ]:
w2v_min.similar_by_word("sale")

[('sal', 0.7034299373626709),
 ('clochard', 0.6563425660133362),
 ('pvte', 0.6445149183273315),
 ('morir?', 0.6245287656784058),
 ('pute', 0.6234241127967834),
 ('chatteux', 0.6222319602966309),
 ('suceur', 0.6207411885261536),
 ('mèrde', 0.6194563508033752),
 ('ntm', 0.6166793704032898),
 ('bdg', 0.6105159521102905)]

In [ ]:
# Encoder une liste de messages 
def encoded(liste):
  w2v=[]
  for i in range(len(liste)):
    w2v.append(np.array([encode(msg.split()) for msg in liste[i]]))
  return w2v

## **Encodage des différents set de clusters**

---

In [ ]:
%%time
#encodage de l'échantillon pour créer le jeu de données
dataset_encoded_km_25 = encoded(clean_cluster_km[0])

CPU times: user 42.8 s, sys: 640 ms, total: 43.5 s
Wall time: 43.8 s


In [ ]:
dataset_encoded_km_50 = encoded(clean_cluster_km[1])

In [ ]:
dataset_encoded_km_100= encoded(clean_cluster_km[2])



## **Entrainement des différents set de clusters**


---

In [ ]:
from sklearn.ensemble import IsolationForest

# definir le modèle ainsi que ses paramètres
clf=IsolationForest(n_estimators=100, max_samples='auto', \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)


In [ ]:
# Entrainer le modèle avec 25 clusters
%%time
clf_km_25= clf
for i in range(len(dataset_encoded_km_25)):
  clf_km_25.fit(dataset_encoded_km_25[i])

CPU times: user 1min 52s, sys: 1.34 s, total: 1min 53s
Wall time: 1min 3s


In [ ]:
%%time
clf_km_50= clf
for i in range(len(dataset_encoded_km_50)):
  clf_km_50.fit(dataset_encoded_km_50[i])

CPU times: user 1min 44s, sys: 2.85 s, total: 1min 47s
Wall time: 1min 6s


In [ ]:
%%time
clf_km_100= clf
for i in range(len(dataset_encoded_km_100)):
  clf_km_100.fit(dataset_encoded_km_100[i])

CPU times: user 1min 43s, sys: 5.08 s, total: 1min 48s
Wall time: 1min 7s




## **Prédictions pour les différents modèles**


---

In [ ]:
#mettre les prédictions dans une liste
%%time
pred=[]
for i in range(len(dataset_encoded_km_25)):
  pred.append(clf_km_25.predict(dataset_encoded_km_25[i]))

CPU times: user 52.6 s, sys: 742 ms, total: 53.3 s
Wall time: 53.3 s


In [ ]:
%%time
pred_50=[]
for i in range(len(dataset_encoded_km_50)):
  pred_50.append(clf_km_50.predict(dataset_encoded_km_50[i]))

CPU times: user 54.6 s, sys: 1 s, total: 55.6 s
Wall time: 57 s


In [ ]:
%%time
pred_100=[]
for i in range(len(dataset_encoded_km_100)):
  pred_100.append(clf_km_100.predict(dataset_encoded_km_100[i]))

CPU times: user 53 s, sys: 1 s, total: 54 s
Wall time: 54.1 s


In [ ]:
pred[1]

array([-1, -1, -1, ..., -1, -1, -1])

In [ ]:
# ajouter une colonne pred à chaque dataframe de cluster
for i in range(len(dataset_encoded_km_25)):
  df[0][i]['pred']= pred[i]

In [ ]:
df[0][3]

,tweet,clean_twt,pred
0,mds ele 😭😭😭\n,mds ele\n,-1
1,QUE DOR MDS 😭😭😭😭😭\n,que dor mds\n,-1
2,mds q agoniante esperar vai logo mds vou infar...,mds q agoniante esperar vai logo mds vou infar...,-1
3,Mds que amor\n,mds que amor\n,-1
4,o george mds https://t.co/mgJdDQNIqM\n,o george mds\n,-1
...,...,...,...
3440,mds mds\n,mds mds\n,-1
3441,"Mds, denovo?\n","mds, denovo?\n",-1
3442,mds amiga que\n,mds amiga que\n,-1
3443,Mds eu to gordaaaa 😥\n,mds eu to gordaaaa\n,-1


In [ ]:
for i in range(len(dataset_encoded_km_50)):
  df[1][i]['pred']= pred_50[i]

In [ ]:
for i in range(len(dataset_encoded_km_100)):
  df[2][i]['pred']= pred_100[i]

## **Isoler les outliers**


---

In [ ]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
def outliers(dataset_encoded, df):
  outliers=[]
  for i in range(len(dataset_encoded)):
    outliers.append(df[i]['tweet'].loc[df[i]['pred']==-1])
    outlier_index=list(outliers[i].index)
    print("cluster"+str(i)+":"+"\n")
    print(df[i]['pred'].value_counts())
    print("\n")
  return outliers

In [ ]:
outliers_25 = outliers(dataset_encoded_km_25, df[0])

cluster0:

 1    30926
-1     1832
Name: pred, dtype: int64


cluster1:

-1    2720
 1      43
Name: pred, dtype: int64


cluster2:

 1    80715
-1     1458
Name: pred, dtype: int64


cluster3:

-1    3438
 1       7
Name: pred, dtype: int64


cluster4:

 1    1446
-1     717
Name: pred, dtype: int64


cluster5:

 1    16987
-1     3842
Name: pred, dtype: int64


cluster6:

 1    40650
-1     1357
Name: pred, dtype: int64


cluster7:

 1    2435
-1    1877
Name: pred, dtype: int64


cluster8:

 1    15169
-1     1262
Name: pred, dtype: int64


cluster9:

-1    1209
 1      85
Name: pred, dtype: int64


cluster10:

 1    54629
-1     1625
Name: pred, dtype: int64


cluster11:

 1    3065
-1    2844
Name: pred, dtype: int64


cluster12:

 1    5650
-1     721
Name: pred, dtype: int64


cluster13:

 1    23026
-1     2605
Name: pred, dtype: int64


cluster14:

-1    3426
 1    3255
Name: pred, dtype: int64


cluster15:

-1    481
Name: pred, dtype: int64


cluster16:

 1    66689
-1     6

In [ ]:
outliers_25[3]

0                                           mds ele 😭😭😭\n
1                                     QUE DOR MDS 😭😭😭😭😭\n
2       mds q agoniante esperar vai logo mds vou infar...
3                                          Mds que amor\n
4                  o george mds https://t.co/mgJdDQNIqM\n
                              ...                        
3440                                            mds mds\n
3441                                       Mds, denovo?\n
3442                                      mds amiga que\n
3443                               Mds eu to gordaaaa 😥\n
3444                                       KKKKKKKK MDS\n
Name: tweet, Length: 3438, dtype: object

In [ ]:
outliers_50 = outliers(dataset_encoded_km_50, df[1])

cluster0:

 1    15344
-1     1481
Name: pred, dtype: int64


cluster1:

 1    2273
-1    1815
Name: pred, dtype: int64


cluster2:

1    13483
Name: pred, dtype: int64


cluster3:

 1    15893
-1     1535
Name: pred, dtype: int64


cluster4:

-1    500
Name: pred, dtype: int64


cluster5:

-1    282
 1    165
Name: pred, dtype: int64


cluster6:

 1    2769
-1    2218
Name: pred, dtype: int64


cluster7:

-1    2783
 1     204
Name: pred, dtype: int64


cluster8:

 1    1918
-1    1184
Name: pred, dtype: int64


cluster9:

 1    13108
-1     1073
Name: pred, dtype: int64


cluster10:

 1    34411
-1      635
Name: pred, dtype: int64


cluster11:

 1    8725
-1     464
Name: pred, dtype: int64


cluster12:

 1    35202
-1     1423
Name: pred, dtype: int64


cluster13:

 1    33437
-1     2523
Name: pred, dtype: int64


cluster14:

 1    6238
-1    1003
Name: pred, dtype: int64


cluster15:

-1    576
 1    407
Name: pred, dtype: int64


cluster16:

 1    29253
-1     1009
Name: pred, d

In [ ]:
outliers_100 = outliers(dataset_encoded_km_100, df[2])

cluster0:

 1    3193
-1     321
Name: pred, dtype: int64


cluster1:

1    13060
Name: pred, dtype: int64


cluster2:

 1    49420
-1      224
Name: pred, dtype: int64


cluster3:

 1    4586
-1    1040
Name: pred, dtype: int64


cluster4:

 1    2407
-1     999
Name: pred, dtype: int64


cluster5:

-1    497
Name: pred, dtype: int64


cluster6:

 1    7868
-1     618
Name: pred, dtype: int64


cluster7:

 1    3981
-1     698
Name: pred, dtype: int64


cluster8:

 1    2202
-1     725
Name: pred, dtype: int64


cluster9:

-1    1195
 1      80
Name: pred, dtype: int64


cluster10:

 1    5420
-1    1221
Name: pred, dtype: int64


cluster11:

 1    14989
-1      317
Name: pred, dtype: int64


cluster12:

 1    7252
-1     209
Name: pred, dtype: int64


cluster13:

 1    5385
-1    1073
Name: pred, dtype: int64


cluster14:

 1    15423
-1      138
Name: pred, dtype: int64


cluster15:

 1    6054
-1    1290
Name: pred, dtype: int64


cluster16:

 1    5757
-1     692
Name: pred, dtype

## **Enregistrer les outliers**


---

In [ ]:
def save_outliers(outliers,num):
  for i in range(len(outliers)):
    with open(f"drive/MyDrive/min/km"+num+f"/outliers{i}", 'w') as f:
      for line in outliers[i]:
        f.write(line)

In [ ]:
save_outliers(outliers_25,"25")

In [ ]:
save_outliers(outliers_50,"50")

In [ ]:
save_outliers(outliers_100,"100")